# Sudo Test Scenarios

Obviously, automated tests that require going through a UAC are challenging to write as unit tests, or run in CI. This notebook instead provides a way of listing a bunch of manual tests. This is all powered by the VsCode Polyglot Notebooks extension. Make sure you have that installed, so that you can run this notebook. 

(You'll need the [Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension for VsCode installed to run this notebook.)

## Building

First, start by building the code. We're going to stick the output exe into a `_sudo_` alias, to keep tests concise. 


In [ ]:
cargo build --target x86_64-pc-windows-msvc
new-alias -Force _sudo_ ..\target\x86_64-pc-windows-msvc\debug\sudo.exe

In [ ]:
new-alias -Force _sudo_ ..\target\x86_64-pc-windows-msvc\debug\sudo.exe

## Tests

### Simple sanity tests
Just start py printing the error message:

In [ ]:
_sudo_

Dismiss this UAC. We should print an error message.

In [ ]:
_sudo_ cmd

Running `sudo notepad` should elevate Notepad directly without triggering a UAC prompt for `notepad`.

In [ ]:
_sudo_ notepad

Running `sudo --newWindow cmd` should elevate "Command Prompt" directly and spawn a new conhost.

In [ ]:
_sudo_ --newWindow cmd

Running `sudo netstat -ab` should display network statistics and the process using them.

In [ ]:
_sudo_ netstat -ab

Exiting `netstat` and `sudo` using Ctrl+C should terminate the processes. This test is going to spawn a new CMD in a new conhost window, ctrl+c in that window to verify it did work. 

In [ ]:
conhost -- cmd.exe /k ..\target\x86_64-pc-windows-msvc\debug\sudo.exe netstat -ab

### Redirecting to pipes and files

This command uses `dir /b` to list the contents of the directory, and `find /c /v ""` to count the lines (items) returned.
This should print a number greater than 0 if there are items in the directory.

In [ ]:
_sudo_ cmd /c dir /b "C:\Program Files\WindowsApps" `| find /c /v ""

### Env vars

Running `sudo cmd`, without -E, doesn't propogate the environment variables. You should get `FooBar was %FooBar%`

In [ ]:
$env:FooBar = "Hello World"
_sudo_ cmd /c echo FooBar was '%FooBar%'

Running `sudo -E ...` should pass the env vars to the child process.

In [ ]:
$env:FooBar = "Hello World"
_sudo_ -E cmd /c echo FooBar was '%FooBar%'

Running `sudo -E` with no command should not crash.

In [ ]:
_sudo_ -E

Running `sudo -N -E cmd` should elevate `sudo` directly, spawn a new conhost, and retain env vars. (The output will appear here in the notebook - close the conhost to continue.)

In [ ]:
$env:FooBar = "Hello new window"
_sudo_ --new-window -E cmd /k echo FooBar was '%FooBar%'

Running `..\target\x86_64-pc-windows-msvc\debug\sudo.exe cmd` from an admin prompt should silently launch CMD without triggering UAC. (you'll need to paste this into the conhost that appears manually).

In [ ]:
Start-Process -verb runas cmd -ArgumentList "/k cd /d $((Get-Location).Path)"

### CMD intrinsics tests

Running `sudo dir`, FROM CMD, should list the files in the current working directory. (Recall, we're in the `/tools` dir)

In [ ]:
cmd /c ..\target\x86_64-pc-windows-msvc\debug\sudo.exe dir

Of course, the notebook is running PowerShell. So, running `sudo dir` from PowerShell should not list files in the current working directory.

In [ ]:
_sudo_ dir

Running the command `target\x86_64-pc-windows-msvc\debug\sudo.exe fsutil volume allocationReport C: > out.txt`, then typing `out.txt` should display content in the text file.

Ensure that the text file is not empty. This usually takes like 15-20 seconds to run, so patience. 

In [ ]:
if (Test-Path -Path "out.txt") { Remove-Item -Path "out.txt" }
_sudo_ fsutil volume allocationReport C: > out.txt
(get-childItem -path out.txt).Length -gt 0
Remove-Item -Path "out.txt"

### Setting Modes


Running `sudo --inline cmd` should exit with an error when `sudo` is set to disable input mode.

In [ ]:
_sudo_ ..\target\x86_64-pc-windows-msvc\debug\sudo.exe config --enable disableInput
_sudo_ --inline cmd

Same as above, but with `sudo --disable-input cmd` should exit with an error when `sudo` is set to force new window mode.

In [ ]:
_sudo_ ..\target\x86_64-pc-windows-msvc\debug\sudo.exe config --enable forceNewWindow
_sudo_ --disable-input cmd

Now, back to normal mode

In [ ]:
_sudo_ ..\target\x86_64-pc-windows-msvc\debug\sudo.exe config --enable normal

### Working directories

Running `sudo -N cmd` in any path should start `cmd` in `C:\windows\system32`. (this will open a conhost in a new window)

In [ ]:
_sudo_ -N cmd

Running `sudo -D . -N cmd` in any path should start in the original path. (the `/tools` directory)

(again, output will actually appear in the notebook, close the conhost to continue)

In [ ]:
_sudo_ -D . -N cmd

This next one is a bit wacky. Search paths are... complicated. 

We're going to copy our built binary into system32 (under a different name, so it doesn't kill your existing sudo). 
Then, we're going to copy `cmd` into a different path, one that's relative to our CWD. 

Then, we're going to run `sudo2 ..\cmd.exe`. This should run the `cmd` from the relative path, not the one in system32. We can verify this by looking at the conhost that's spawned. It should display an error at the start like:

```
The system cannot find message text for message number 0x2350 in the message file for Application.

(c) Microsoft Corporation. All rights reserved.
Not enough memory resources are available to process this command.
```

There will be two UACs to accept. 

In [ ]:
_sudo_ cmd /c copy ..\target\x86_64-pc-windows-msvc\debug\sudo.exe C:\Windows\System32\sudo2.exe /y
copy-item c:\windows\system32\cmd.exe -destination .. -force
c:\windows\system32\sudo2.exe -N ..\cmd.exe
rm ..\cmd.exe